In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [2]:
response = requests.get(
    "https://attack.mitre.org/techniques/enterprise/")
soup = BeautifulSoup(response.text, "html.parser")
# print(soup.prettify())  

In [3]:
soup_techniques = soup.find_all("tr", class_="technique")

In [4]:
len(soup_techniques)

625

In [12]:
# {"T1057":["xxxx","cccc"]}
techniques = {}
# {"T1057":["xxxx","cccc"]}
sub_techniques_links = {}
technique_id  = ""
for soup_technique in soup_techniques:
    temp_id = soup_technique.select("td")[0].text.strip()
    if temp_id != "":
        technique_id = temp_id
        technique_description = soup_technique.select("td")[2].text.strip()
        link = soup_technique.select("td")[1].find("a").get("href")
        techniques[technique_id] = [technique_description]
        sub_techniques_links[technique_id] = [link]
    else:
        # technique_id = soup_technique.select("td")[1].text.strip()
        technique_description = soup_technique.select("td")[3].text.strip()
        link = soup_technique.select("td")[2].find("a").get("href")
        techniques[technique_id].append(technique_description)
        sub_techniques_links[technique_id].append(link)

In [15]:
len(techniques)
techniques_id = []
for key, value in techniques.items():
    techniques_id.append(key)

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


mitigations_data = []
for technique_id in techniques_id:
    url = f"https://attack.mitre.org/techniques/{technique_id}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    mitigation_section = soup.find('h2', id='mitigations').find_next('table')
    
    for row in mitigation_section.find_all('tr')[1:]:  # Skipping the header row
        cols = row.find_all('td')
        if len(cols) > 2:  # Ensure there are enough columns
            mitigation_id = cols[0].text.strip()
            mitigation_name = cols[1].text.strip()
            mitigation_description = cols[2].text.strip()  # Getting the description directly
            if mitigation_id == "" or mitigation_name == "" or mitigation_description == "" or mitigation_id[:2]=="DS":
                continue
            mitigations_data.append({
                'TechniqueID': technique_id,
                'MitigationID': mitigation_id,
                'MitigationName': mitigation_name,
                'MitigationDescription': mitigation_description
            })
            




# Convert to DataFrame
df = pd.DataFrame(mitigations_data)
print(df)
# df.to_csv('/mnt/data/mitigations.csv', index=False)  # Save the DataFrame as a CSV

# df.head()  # Display the first few rows of the DataFrame


    TechniqueID MitigationID                           MitigationName  \
0         T1548        M1047                                    Audit   
1         T1548        M1038                     Execution Prevention   
2         T1548        M1028           Operating System Configuration   
3         T1548        M1026            Privileged Account Management   
4         T1548        M1022  Restrict File and Directory Permissions   
..          ...          ...                                      ...   
463       T1047        M1040          Behavior Prevention on Endpoint   
464       T1047        M1038                     Execution Prevention   
465       T1047        M1026            Privileged Account Management   
466       T1047        M1018                  User Account Management   
467       T1220        M1038                     Execution Prevention   

                                 MitigationDescription  
0    Check for common UAC bypass weaknesses on Wind...  
1    Syst

In [24]:
df.to_excel('../RAG/mitigations.xlsx', index=False)  # Save the DataFrame as a CSV